In [11]:
from credentials import *
import openai
import json
import time
from requests import post

In [3]:
openai.api_key = OPENAI_API_KEY

In [6]:
instructions = json.loads(open('instructions.json').read())

In [14]:
def make_prompt(key=None, model=None) -> str:
    if key == None:
        raise TypeError('Key is None')
    if model == None:
        raise Exception('Provide a model name')
    question = instructions[key][model]['Response']
    answer = instructions[key]['instruction']
    prompt = "Evaluate answers provided by language model to instructions. " + \
            "If the answer is false explain why. Possible reasons: factual mistake, grammar mistake, logic mistake, no common sense, not according to the instruction, unethical, should not answer as a language model." +  \
            "Use this template to say if answer is false: <listed reasons from the list of possible reasons and brief explanation>; template if answer is true: 'TRUE'." + \
            "Example of your answer if True: TRUE. Example of your answer if False: logic mistake, no common sense. " + \
            f"Question: '{question}'. Answer: '{answer}'. "
    return prompt

In [18]:
def make_open_ai_request(prompt=None, token=OPENAI_API_KEY):
    if prompt == None:
        raise Exception('Provide prompt')
    url = 'https://api.openai.com/v1/chat/completions'
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    body = {
        'model' : 'gpt-3.5-turbo',
        'messages' : [{'role' : 'user', 'content' : prompt}],
        'temperature' : 0.7
    }
    return post(url, headers=headers, json=body)

In [19]:
x = make_open_ai_request(prompt=make_prompt(key='1', model='Yandex_GPT'))

In [26]:
x.json()['choices'][0]['message']['content']

'FALSE. The answer is not according to the instruction. The model was asked to describe the process of selecting a candidate for a vacant position, but it only provided a summary of the steps involved in the process.'

In [37]:
import nltk
def get_eval(response:json) -> (str, str):
    response_str = response['choices'][0]['message']['content']
    tokens = nltk.word_tokenize(response_str)
    evl = tokens[0].lower()
    if evl == 'true':
        res = 'TRUE'
        comment = ''
    else:
        res = 'FALSE'
        comment = response_str[7:]
    return res, comment

In [40]:
get_eval(x.json())

('FALSE',
 'The answer is not according to the instruction. The model was asked to describe the process of selecting a candidate for a vacant position, but it only provided a summary of the steps involved in the process.')

In [49]:
for k in instructions.keys():
    if instructions[k]['Gigachat']['Eval'] != None and instructions[k]['Yandex_GPT']['Eval'] != None:
        print(k)
        continue
    print(k)
    ya_prompt = make_prompt(key=k, model='Yandex_GPT')
    giga_prompt = make_prompt(key=k, model='Gigachat')
    ya_response = make_open_ai_request(ya_prompt)
    try: 
        evaluation, comment = get_eval(ya_response.json())
    except:
        print('Unable to extract eval and comment')
        continue
    instructions[k]['Yandex_GPT']['Comment'] = comment
    instructions[k]['Yandex_GPT']['Eval'] = evaluation
    time.sleep(2)
    giga_response = make_open_ai_request(giga_prompt)
    try:
        evaluation, comment = get_eval(giga_response.json())
    except:
        print('Unable to extract eval and comment')
        continue
    instructions[k]['Gigachat']['Comment'] = comment
    instructions[k]['Gigachat']['Eval'] = evaluation
    time.sleep(2)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194


In [50]:
with open('instructions.json', 'w', encoding='utf-8') as w:
    w.write(json.dumps(instructions))

In [51]:
instructions

{'1': {'instruction': 'Опиши процесс выбора кандидата на вакансию.',
  'type': 'hr',
  'Yandex_GPT': {'Response': 'Процесс выбора кандидата на вакантное место представляет собой сложную и многогранную задачу, которая требует тщательного анализа и оценки потенциальных кандидатов. Этот процесс обычно включает в себя следующие шаги:\n\n1. Определение требований к кандидату: Прежде всего, необходимо четко определить требования к кандидату на вакантную должность. Это может включать в себя профессиональные навыки, опыт работы, образование, личные качества и другие аспекты, которые важны для успешного выполнения работы.\n\n2. Поиск кандидатов: После определения требований, можно начать поиск кандидатов. Это может быть осуществлено через различные каналы, такие как интернет-ресурсы, профессиональные сети, кадровые агентства или напрямую от работодателей.\n\n3. Оценка кандидатов: Проведя предварительный отбор, следует провести более тщательную оценку кандидатов. Для этого можно использовать раз